In [1]:
import pandas as pd

In [2]:
df = pd.read_parquet('finodays_tinkoff_payments.gz')

In [ ]:
df[:100000]

In [3]:

df.date_period = pd.to_datetime(df.date)
df.info()
df.date_period = df.date_period.dt.to_period('M')
df

C:\Users\ivlad\AppData\Local\Temp\ipykernel_14560\2811215086.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  df.date_period = pd.to_datetime(df.date)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 64115078 entries, 0 to 64115077
Data columns (total 4 columns):
 #   Column  Dtype 
---  ------  ----- 
 0   payer   int64 
 1   date    object
 2   amount  int64 
 3   phone   int64 
dtypes: int64(3), object(1)
memory usage: 1.9+ GB


,payer,date,amount,phone
0,256082,2020-09-25,100,79217988811
1,577125,2020-09-25,100,79211031739
2,332492,2020-09-25,2000,79220885615
3,1093977,2020-09-25,100,79215570961
4,1037953,2020-09-25,10000,79221073575
...,...,...,...,...
64115073,421252,2021-03-23,1000,79220416876
64115074,765025,2021-03-23,5000,79220751544
64115075,230665,2021-03-23,4000,79216171959
64115076,115111,2021-03-23,500,79215595398


In [5]:
user_df = pd.DataFrame()
user_df = df.loc[df['payer']==334850]

In [21]:
user_df 

,payer,date,amount,phone
934124,230665,2020-09-26,11000,79216171959
14664298,230665,2020-10-30,3000,79213708625
15148818,230665,2020-10-31,500,79220865086
15473470,230665,2020-11-01,11000,79216171959
22778982,230665,2020-11-21,1000,79219084889
24176803,230665,2020-11-25,11000,79216171959
27608227,230665,2020-12-04,2000,79218619787
27682385,230665,2020-12-04,2000,79210440111
30952105,230665,2020-12-13,4500,79211426162
34344565,230665,2020-12-23,3000,79213708625


In [4]:
from numpy import array
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.utils import plot_model
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
# define input sequence
config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

seq_in = np.array([[i for i in range(len(user_df.date))], user_df.amount.to_list(), user_df.amount.values, user_df.phone.to_list()], dtype=float)
# seq_in = user_df.to_numpy()
print(seq_in)
# reshape input into [samples, timesteps, features]
n_in = len(seq_in[0])
# print(n_in)
seq_in = seq_in.reshape((len(seq_in), n_in, 1))
seq_in_in = seq_in[:-2, :-1, :]
# prepare output sequence
seq_out = seq_in[-2:, -1:, :]
n_out = 2
# define model
model = Sequential()
model.add(LSTM(100, activation='relu', input_shape=(n_in-1,1)))
model.add(RepeatVector(n_out))
model.add(LSTM(100, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(1)))
model.compile(optimizer='adam', loss='mse')
plot_model(model, show_shapes=True, to_file='predict_lstm_autoencoder.png')
# fit model
model.fit(seq_in_in, seq_out, epochs=300, verbose=0)
# demonstrate prediction
yhat = model.predict(seq_in, verbose=0)
print(yhat[:,:,:])

[[0.00000000e+00 1.00000000e+00 2.00000000e+00 ... 1.36400000e+03
  1.36500000e+03 1.36600000e+03]
 [1.00000000e+03 1.50000000e+04 5.00000000e+02 ... 1.00000000e+03
  5.00000000e+02 5.00000000e+03]
 [1.00000000e+03 1.50000000e+04 5.00000000e+02 ... 1.00000000e+03
  5.00000000e+02 5.00000000e+03]
 [7.92180525e+10 7.92128733e+10 7.92128462e+10 ... 7.92153473e+10
  7.92180328e+10 7.92128733e+10]]
('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')
[[[nan]
  [nan]]

 [[nan]
  [nan]]

 [[nan]
  [nan]]

 [[nan]
  [nan]]]


In [8]:
# user_df.set_index('date', inplace=True)
# user_df.index = pd.to_datetime(user_df.index)
# user_df.index = user_df.index.to_period('M')
phones = pd.unique(user_df.phone)
new_rent = pd.DataFrame()
uniq_date_list = user_df.index.unique()
new_rent['uniq_date'] = uniq_date_list
new_rent.set_index('uniq_date', inplace=True)
# new_rent['count'] = 0
# new_rent['days_in_month'] = 0

new_rent['year'] = new_rent.index.year.astype(int)
new_rent['month'] = new_rent.index.month.astype(int)
new_rent['day'] = new_rent.index.day.astype(int)
new_rent['dayofweek'] = new_rent.index.dayofweek

for i in phones:
    new_rent[i]=0
for date_rent in uniq_date_list:

    # print(user_df.loc[user_df.index == date_rent, 'phone'])
    # print(date_rent)
    for phone in phones:
        # print(phone)
    
        
        # print(phone)
        # print(user_df.loc[user_df.index == date_rent, 'phone'].to_list())
        if phone in user_df.loc[user_df.index == date_rent, 'phone'].to_list():
            new_rent[phone][date_rent] = 1
        else:
            new_rent[phone][date_rent] = 0
        # new_rent[phone][date_rent] = user_df[user_df.loc[user_df.index == date_rent]].loc[user_df['phone'] == phone]
    # p = pd.Period(date_rent)
    # new_rent['days_in_month'][date_rent] /= p.days_in_month
new_rent = new_rent.sort_index()
new_rent.to_csv('test_334850.csv')
new_rent

,year,month,day,dayofweek,79218520455,79220443028,79217777995,79219985251,79213935652,79220535973,...,79217841806,79212948150,79214033997,79214436267,79217833613,79218262733,79215539911,79217551383,79211907696,79218306210
uniq_date,,,,,,,,,,,,,,,,,,,,,
2020-09-26,2020,9,26,5,1,1,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-10-01,2020,10,1,3,0,0,0,1,1,1,...,0,0,0,0,0,0,0,0,0,0
2020-10-02,2020,10,2,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-10-03,2020,10,3,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2020-10-04,2020,10,4,6,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-03-19,2021,3,19,4,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-03-20,2021,3,20,5,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2021-03-21,2021,3,21,6,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [36]:
import xgboost as xgb
# from xgboost import XGBClassifier
X_train = new_rent.loc[:, 'year':'dayofweek'][:-2]

X_test = new_rent.loc[:,'year':'dayofweek'][-2:-1]
for ph in phones:
    y_train = new_rent[ph][:-2]
    y_test = new_rent[ph][-2:-1]

    dtrain = xgb.DMatrix(X_train, label=y_train)
    dtest = xgb.DMatrix(X_test)

    params = {
            'objective': 'reg:squarederror',
            # 'booster':'gblinear'
        }

    trees = 30
    cv = xgb.cv(params, dtrain, metrics = ('rmse'), verbose_eval=False, nfold=10, show_stdv=False, num_boost_round=trees)
    print(cv['test-rmse-mean'].argmin())
    bst = xgb.train(params, dtrain, num_boost_round=cv['test-rmse-mean'].argmin())

    prediction_test = bst.predict(dtest)
    print('prediction_last', prediction_test)
    if prediction_test > 0.6 and prediction_test < 1:
        prediction_test = 1
    else:
        prediction_test = 0
    # clf = xgb.XGBClassifier(tree_method="hist")
    # clf.fit(X_train, y_train)

    # prediction_test = clf.predict(X_test)
    print('y_test', list(y_test))
    print('prediction', prediction_test)

5
prediction_last [0.0862818]
y_test [0]
prediction 0
9
prediction_last [0.01870356]
y_test [0]
prediction 0
4
prediction_last [0.14728804]
y_test [0]
prediction 0
11
prediction_last [0.01034363]
y_test [0]
prediction 0
4
prediction_last [0.12261098]
y_test [0]
prediction 0
4
prediction_last [0.14416309]
y_test [0]
prediction 0
5
prediction_last [0.37663925]
y_test [0]
prediction 0
2
prediction_last [0.6299327]
y_test [0]
prediction 1
7
prediction_last [0.04308595]
y_test [0]
prediction 0
28
prediction_last [-0.00061231]
y_test [0]
prediction 0
2
prediction_last [0.6913014]
y_test [1]
prediction 1
5
prediction_last [0.09373158]
y_test [0]
prediction 0
2
prediction_last [0.26545453]
y_test [0]
prediction 0
2
prediction_last [0.2608062]
y_test [0]
prediction 0
9
prediction_last [0.0212073]
y_test [0]
prediction 0
10
prediction_last [0.0147635]
y_test [0]
prediction 0
4
prediction_last [0.5464405]
y_test [0]
prediction 0
4
prediction_last [0.40969706]
y_test [0]
prediction 0
15
prediction

In [78]:
! pip install xgboost

You should consider upgrading via the 'c:\users\ivlad\appdata\local\programs\python\python38\python.exe -m pip install --upgrade pip' command.


In [13]:
from predict_nums import pred

In [14]:

pred(265865)


(1, 71)


NameError: name 'model' is not defined